In [1]:
K_q = lambda K_mathcal , q : [idx for idx,K in enumerate(K_mathcal) if q in K]

prob_K_q = lambda K_mathcal, q, prob : sum([prob[idx] for idx in K_q(K_mathcal , q)])

In [9]:
import numpy as np

# Q is the set of items
# K_mathcal is the knowledge network
# K0 is the set of items that the student understands. We use it for simulations but not for real student.
# beta_q is the probability for the student to fail to response item q despite the fact that he manages this item.

class knowledge_learning_space:
    
    def __init__(self,Q,K_mathcal,Xi,beta_q=None,prob_distribution = None,K0=None,student_name = None):

        
        if student_name != None:
            self.student_name = student_name
        
        if (beta_q == None) and (K0!= None):
            self.beta_q = {q : 0.0 for q in K0}
        else:    
            self.beta_q = beta_q
        
        self.K0 = K0
        self.Xi = Xi
        self.Q = Q
        self.K_mathcal = K_mathcal
        n_K = len(K_mathcal)
        
        if prob_distribution == None:
            
            self.prob_distribution =[1/n_K for K in K_mathcal] 
            
        else:
            
            self.prob_distribution = prob_distribution
            
        self.prob_distribution_items = {q : prob_K_q(K_mathcal, q,self.prob_distribution) for q in self.Q}    
        self.min_half = min([np.abs(p - 0.5) for p in self.prob_distribution_items.values()])    
        self.distance_half = [q for q in self.Q if np.abs(self.prob_distribution_items[q]  -0.5 ) \
                              == self.min_half]
        
        self.responded_items = {q : [] for q in Q}
        
        idx_next_item = np.random.randint(0,len(self.distance_half))
        
        self.next_item = self.distance_half[idx_next_item]
        
    def simulation(self):
        
        if self.K0 == None:
            raise NameError('No se puede simular sin conocer K0')
        
        q = self.next_item
        r = int((q in self.K0) and (np.random.rand() <= 1 - self.beta_q[q]))
        self.responded_items[q].append(r)
        
        for idx,K in enumerate(self.K_mathcal):
            
            if int(q in K) == r:
                
                self.prob_distribution[idx] = self.Xi[(q,r)] * self.prob_distribution[idx]
                
        S = np.sum(self.prob_distribution)        
                
        for idx in range(len(self.K_mathcal)):     
                
                self.prob_distribution[idx] = self.prob_distribution[idx] / S
                
        self.prob_distribution_items = {q : prob_K_q(K_mathcal, q,self.prob_distribution) for q in self.Q}    
        
        self.min_half = min([np.abs(p - 0.5) for p in self.prob_distribution_items.values()])    
            
        self.distance_half = [q for q in self.Q if np.abs(self.prob_distribution_items[q]  -0.5 ) \
                              == self.min_half]
        
        idx_next_item = np.random.randint(0,len(self.distance_half))
        
        self.next_item = self.distance_half[idx_next_item]
        
    def response_item(self,r):
        
        q = self.next_item
        self.responded_items[q].append(r)    
        
        for idx,K in enumerate(self.K_mathcal):
            
            if int(q in K) == r:
                
                self.prob_distribution[idx] = self.Xi[(q,r)] * self.prob_distribution[idx]
                
        S = np.sum(self.prob_distribution)        
                
        for idx in range(len(self.K_mathcal)):     
                
                self.prob_distribution[idx] = self.prob_distribution[idx] / S
                
        self.prob_distribution_items = {q : prob_K_q(K_mathcal, q,self.prob_distribution) for q in Q}    
        self.min_half = min([np.abs(p - 0.5) for p in self.prob_distribution_items.values()])    
        self.distance_half = [q for q in self.Q if np.abs(self.prob_distribution_items[q]  -0.5 ) \
                              == self.min_half]
        
        idx_next_item = np.random.randint(0,len(self.distance_half))
        self.next_item = self.distance_half[idx_next_item]

# Example 1: simulación

In [148]:
Q = ['a' , 'b' , 'c' , 'd' , 'e']
K_mathcal = [[],['a'],['b'],['a','b'],['b','d'],['a','b','c'],['a','b','d'],['b','c','d'],Q]
K0 = ['b','d']
beta_q = {'b' : 0.1 ,'d' : 0.1}

QxR = []

for q in Q:
    for r in [0,1]:
        QxR.append((q,r))

Xi = {x : 4.0 for x in QxR}

initial_prob = []
S = 0
for K in K_mathcal[:-1]:
    p = (1 - S) * np.random.rand()
    S += p
    initial_prob.append(p)
initial_prob.append(1-S)

In [149]:
import random

initial_prob = sorted(initial_prob, key=lambda y: random.randint(0, len(initial_prob)))

In [150]:
network_learning = knowledge_learning_space(Q,K_mathcal,Xi,beta_q,initial_prob,K0)

In [151]:
network_learning.responded_items

{'a': [], 'b': [], 'c': [], 'd': [], 'e': []}

Probabilidad de cada estado de conocimiento:

In [152]:
for knowledge_state, prob in zip(network_learning.K_mathcal,network_learning.prob_distribution):
    print(f'Probability({knowledge_state})={prob}')

Probability([])=0.036148110402929826
Probability(['a'])=0.020047230884785636
Probability(['b'])=0.011649976781145653
Probability(['a', 'b'])=0.14090141881614154
Probability(['b', 'd'])=0.10072915756836243
Probability(['a', 'b', 'c'])=0.0013343470014371972
Probability(['a', 'b', 'd'])=0.6594923857549356
Probability(['b', 'c', 'd'])=0.011484242716702288
Probability(['a', 'b', 'c', 'd', 'e'])=0.018213130073559597


Probabilidad que el estudiante maneje cada item:

In [153]:
for item, prob in network_learning.prob_distribution_items.items():
    print(f'Probability({item})={prob}')

Probability(a)=0.8399885125308596
Probability(b)=0.9438046587122845
Probability(c)=0.03103171979169908
Probability(d)=0.78991891611356
Probability(e)=0.018213130073559597


In [154]:
network_learning.next_item

'd'

In [155]:
network_learning.simulation()

In [156]:
network_learning.responded_items

{'a': [], 'b': [], 'c': [], 'd': [1], 'e': []}

In [157]:
for knowledge_state, prob in zip(network_learning.K_mathcal,network_learning.prob_distribution):
    print(f'Probability({knowledge_state})={prob}')

Probability([])=0.010727216562658331
Probability(['a'])=0.005949162619722388
Probability(['b'])=0.0034572159509383815
Probability(['a', 'b'])=0.041813528197702565
Probability(['b', 'd'])=0.11956846157273875
Probability(['a', 'b', 'c'])=0.00039597724734708226
Probability(['a', 'b', 'd'])=0.7828367861623006
Probability(['b', 'c', 'd'])=0.013632132612963596
Probability(['a', 'b', 'c', 'd', 'e'])=0.021619519073628114


In [158]:
for item, prob in network_learning.prob_distribution_items.items():
    print(f'Probability({item})={prob}')

Probability(a)=0.8526149733007007
Probability(b)=0.983323620817619
Probability(c)=0.035647628933938795
Probability(d)=0.937656899421631
Probability(e)=0.021619519073628114


In [159]:
network_learning.next_item

'a'

In [160]:
network_learning.simulation()

In [161]:
network_learning.responded_items

{'a': [0], 'b': [], 'c': [], 'd': [1], 'e': []}

In [162]:
for knowledge_state, prob in zip(network_learning.K_mathcal,network_learning.prob_distribution):
    print(f'Probability({knowledge_state})={prob}')

Probability([])=0.02975329549698675
Probability(['a'])=0.004125189240617968
Probability(['b'])=0.009589026863057466
Probability(['a', 'b'])=0.02899378074850602
Probability(['b', 'd'])=0.3316382911181012
Probability(['a', 'b', 'c'])=0.0002745732777366789
Probability(['a', 'b', 'd'])=0.5428242752569714
Probability(['b', 'c', 'd'])=0.0378104485463192
Probability(['a', 'b', 'c', 'd', 'e'])=0.01499111945170316


In [163]:
for item, prob in network_learning.prob_distribution_items.items():
    print(f'Probability({item})={prob}')

Probability(a)=0.5912089379755352
Probability(b)=0.9661215152623951
Probability(c)=0.05307614127575904
Probability(d)=0.9272641343730951
Probability(e)=0.01499111945170316


In [164]:
network_learning.simulation()

In [165]:
network_learning.responded_items

{'a': [0, 0], 'b': [], 'c': [], 'd': [1], 'e': []}

In [166]:
for knowledge_state, prob in zip(network_learning.K_mathcal,network_learning.prob_distribution):
    print(f'Probability({knowledge_state})={prob}')

Probability([])=0.05345607948047918
Probability(['a'])=0.0018528741122208156
Probability(['b'])=0.017228067465130455
Probability(['a', 'b'])=0.0130228754684392
Probability(['b', 'd'])=0.5958359419572502
Probability(['a', 'b', 'c'])=0.0001233276071838333
Probability(['a', 'b', 'd'])=0.24381549268222139
Probability(['b', 'c', 'd'])=0.06793191506767052
Probability(['a', 'b', 'c', 'd', 'e'])=0.006733426159404423


In [167]:
for item, prob in network_learning.prob_distribution_items.items():
    print(f'Probability({item})={prob}')

Probability(a)=0.26554799602946966
Probability(b)=0.9446910464073
Probability(c)=0.07478866883425878
Probability(d)=0.9143167758665465
Probability(e)=0.006733426159404423


In [168]:
network_learning.simulation()

In [169]:
network_learning.responded_items

{'a': [0, 0, 0], 'b': [], 'c': [], 'd': [1], 'e': []}

In [170]:
for knowledge_state, prob in zip(network_learning.K_mathcal,network_learning.prob_distribution):
    print(f'Probability({knowledge_state})={prob}')

Probability([])=0.06675009494006189
Probability(['a'])=0.0005784165435658584
Probability(['b'])=0.021512522992846703
Probability(['a', 'b'])=0.004065384996239567
Probability(['b', 'd'])=0.7440146393240722
Probability(['a', 'b', 'c'])=3.8499500750226636e-05
Probability(['a', 'b', 'd'])=0.07611251817643752
Probability(['b', 'c', 'd'])=0.08482593232231143
Probability(['a', 'b', 'c', 'd', 'e'])=0.0021019912037145926


In [171]:
for item, prob in network_learning.prob_distribution_items.items():
    print(f'Probability({item})={prob}')

Probability(a)=0.08289681042070778
Probability(b)=0.9326714885163722
Probability(c)=0.08696642302677626
Probability(d)=0.9070550810265356
Probability(e)=0.0021019912037145926


In [172]:
network_learning.simulation()

In [173]:
network_learning.responded_items

{'a': [0, 0, 0], 'b': [], 'c': [], 'd': [1, 1], 'e': []}

In [174]:
for knowledge_state, prob in zip(network_learning.K_mathcal,network_learning.prob_distribution):
    print(f'Probability({knowledge_state})={prob}')

Probability([])=0.01793795507044456
Probability(['a'])=0.00015543962865974893
Probability(['b'])=0.005781125423778039
Probability(['a', 'b'])=0.0010925032162439224
Probability(['b', 'd'])=0.7997652248394448
Probability(['a', 'b', 'c'])=1.0346087377287432e-05
Probability(['a', 'b', 'd'])=0.08181578962986057
Probability(['b', 'c', 'd'])=0.09118211826799733
Probability(['a', 'b', 'c', 'd', 'e'])=0.0022594978361939137


In [175]:
for item, prob in network_learning.prob_distribution_items.items():
    print(f'Probability({item})={prob}')

Probability(a)=0.08533357639833544
Probability(b)=0.9819066053008959
Probability(c)=0.09345196219156854
Probability(d)=0.9750226305734967
Probability(e)=0.0022594978361939137


In [176]:
network_learning.simulation()

In [177]:
network_learning.responded_items

{'a': [0, 0, 0], 'b': [], 'c': [0], 'd': [1, 1], 'e': []}

In [178]:
for knowledge_state, prob in zip(network_learning.K_mathcal,network_learning.prob_distribution):
    print(f'Probability({knowledge_state})={prob}')

Probability([])=0.019289969172804653
Probability(['a'])=0.00016715537714882063
Probability(['b'])=0.006216858653667698
Probability(['a', 'b'])=0.0011748470368987778
Probability(['b', 'd'])=0.860044886501755
Probability(['a', 'b', 'c'])=2.7814723833243465e-06
Probability(['a', 'b', 'd'])=0.08798238448088429
Probability(['b', 'c', 'd'])=0.02451366729921664
Probability(['a', 'b', 'c', 'd', 'e'])=0.0006074500052407482


In [179]:
for item, prob in network_learning.prob_distribution_items.items():
    print(f'Probability({item})={prob}')

Probability(a)=0.08993461837255595
Probability(b)=0.9805428754500465
Probability(c)=0.025123898776840712
Probability(d)=0.9731483882870967
Probability(e)=0.0006074500052407482


In [180]:
network_learning.simulation()

In [181]:
network_learning.responded_items

{'a': [0, 0, 0, 0], 'b': [], 'c': [0], 'd': [1, 1], 'e': []}

In [182]:
for knowledge_state, prob in zip(network_learning.K_mathcal,network_learning.prob_distribution):
    print(f'Probability({knowledge_state})={prob}')

Probability([])=0.020685206271814586
Probability(['a'])=4.4811417592115254e-05
Probability(['b'])=0.00666652198672926
Probability(['a', 'b'])=0.00031495583375973867
Probability(['b', 'd'])=0.9222516490793112
Probability(['a', 'b', 'c'])=7.45663840530318e-07
Probability(['a', 'b', 'd'])=0.0235865303226996
Probability(['b', 'c', 'd'])=0.02628673275838144
Probability(['a', 'b', 'c', 'd', 'e'])=0.0001628466658714833


# Example 2: response of student

In [6]:
Q = ['a' , 'b' , 'c' , 'd' , 'e']
K_mathcal = [[],['a'],['b'],['a','b'],['b','d'],['a','b','c'],['a','b','d'],['b','c','d'],Q]
nombre = "Paula"

QxR = []

for q in Q:
    for r in [0,1]:
        QxR.append((q,r))

Xi = {x : 4.0 for x in QxR}

In [10]:
Paula =  knowledge_learning_space(Q,K_mathcal,Xi,student_name = nombre)

In [11]:
Paula.responded_items

{'a': [], 'b': [], 'c': [], 'd': [], 'e': []}

Asumimos que Paula sabe responder correctamente a los items a y b

In [13]:
for knowledge_state, prob in zip(Paula.K_mathcal,Paula.prob_distribution):
    print(f'Probability({knowledge_state})={prob}')

Probability([])=0.1111111111111111
Probability(['a'])=0.1111111111111111
Probability(['b'])=0.1111111111111111
Probability(['a', 'b'])=0.1111111111111111
Probability(['b', 'd'])=0.1111111111111111
Probability(['a', 'b', 'c'])=0.1111111111111111
Probability(['a', 'b', 'd'])=0.1111111111111111
Probability(['b', 'c', 'd'])=0.1111111111111111
Probability(['a', 'b', 'c', 'd', 'e'])=0.1111111111111111


In [14]:
for item, prob in Paula.prob_distribution_items.items():
    print(f'Probability({item})={prob}')

Probability(a)=0.5555555555555556
Probability(b)=0.7777777777777779
Probability(c)=0.3333333333333333
Probability(d)=0.4444444444444444
Probability(e)=0.1111111111111111


In [15]:
Paula.next_item

'a'

In [16]:
Paula.response_item(1)

In [17]:
Paula.responded_items

{'a': [1], 'b': [], 'c': [], 'd': [], 'e': []}

In [18]:
for knowledge_state, prob in zip(Paula.K_mathcal,Paula.prob_distribution):
    print(f'Probability({knowledge_state})={prob}')

Probability([])=0.04166666666666666
Probability(['a'])=0.16666666666666663
Probability(['b'])=0.04166666666666666
Probability(['a', 'b'])=0.16666666666666663
Probability(['b', 'd'])=0.04166666666666666
Probability(['a', 'b', 'c'])=0.16666666666666663
Probability(['a', 'b', 'd'])=0.16666666666666663
Probability(['b', 'c', 'd'])=0.04166666666666666
Probability(['a', 'b', 'c', 'd', 'e'])=0.16666666666666663


In [19]:
for item, prob in Paula.prob_distribution_items.items():
    print(f'Probability({item})={prob}')

Probability(a)=0.8333333333333331
Probability(b)=0.7916666666666665
Probability(c)=0.3749999999999999
Probability(d)=0.4166666666666666
Probability(e)=0.16666666666666663


In [20]:
Paula.next_item

'd'

In [21]:
Paula.response_item(0)

In [22]:
Paula.responded_items

{'a': [1], 'b': [], 'c': [], 'd': [0], 'e': []}

In [23]:
for knowledge_state, prob in zip(Paula.K_mathcal,Paula.prob_distribution):
    print(f'Probability({knowledge_state})={prob}')

Probability([])=0.060606060606060615
Probability(['a'])=0.24242424242424246
Probability(['b'])=0.060606060606060615
Probability(['a', 'b'])=0.24242424242424246
Probability(['b', 'd'])=0.015151515151515154
Probability(['a', 'b', 'c'])=0.24242424242424246
Probability(['a', 'b', 'd'])=0.060606060606060615
Probability(['b', 'c', 'd'])=0.015151515151515154
Probability(['a', 'b', 'c', 'd', 'e'])=0.060606060606060615


In [24]:
for item, prob in Paula.prob_distribution_items.items():
    print(f'Probability({item})={prob}')

Probability(a)=0.8484848484848487
Probability(b)=0.6969696969696971
Probability(c)=0.31818181818181823
Probability(d)=0.15151515151515155
Probability(e)=0.060606060606060615


In [25]:
Paula.next_item

'c'

In [26]:
Paula.response_item(0)

In [27]:
Paula.responded_items

{'a': [1], 'b': [], 'c': [0], 'd': [0], 'e': []}

In [28]:
for knowledge_state, prob in zip(Paula.K_mathcal,Paula.prob_distribution):
    print(f'Probability({knowledge_state})={prob}')

Probability([])=0.07960199004975124
Probability(['a'])=0.31840796019900497
Probability(['b'])=0.07960199004975124
Probability(['a', 'b'])=0.31840796019900497
Probability(['b', 'd'])=0.01990049751243781
Probability(['a', 'b', 'c'])=0.07960199004975124
Probability(['a', 'b', 'd'])=0.07960199004975124
Probability(['b', 'c', 'd'])=0.004975124378109453
Probability(['a', 'b', 'c', 'd', 'e'])=0.01990049751243781


In [29]:
for item, prob in Paula.prob_distribution_items.items():
    print(f'Probability({item})={prob}')

Probability(a)=0.8159203980099502
Probability(b)=0.6019900497512437
Probability(c)=0.1044776119402985
Probability(d)=0.12437810945273631
Probability(e)=0.01990049751243781


In [30]:
Paula.response_item(1)

In [31]:
Paula.responded_items

{'a': [1], 'b': [1], 'c': [0], 'd': [0], 'e': []}

In [32]:
for knowledge_state, prob in zip(Paula.K_mathcal,Paula.prob_distribution):
    print(f'Probability({knowledge_state})={prob}')

Probability([])=0.028368794326241138
Probability(['a'])=0.11347517730496455
Probability(['b'])=0.11347517730496455
Probability(['a', 'b'])=0.4539007092198582
Probability(['b', 'd'])=0.028368794326241138
Probability(['a', 'b', 'c'])=0.11347517730496455
Probability(['a', 'b', 'd'])=0.11347517730496455
Probability(['b', 'c', 'd'])=0.007092198581560284
Probability(['a', 'b', 'c', 'd', 'e'])=0.028368794326241138


In [33]:
for item, prob in Paula.prob_distribution_items.items():
    print(f'Probability({item})={prob}')

Probability(a)=0.8226950354609931
Probability(b)=0.8581560283687946
Probability(c)=0.14893617021276598
Probability(d)=0.17730496453900713
Probability(e)=0.028368794326241138


In [34]:
Paula.next_item

'd'

In [35]:
Paula.response_item(0)

In [36]:
for knowledge_state, prob in zip(Paula.K_mathcal,Paula.prob_distribution):
    print(f'Probability({knowledge_state})={prob}')

Probability([])=0.032719836400818
Probability(['a'])=0.130879345603272
Probability(['b'])=0.130879345603272
Probability(['a', 'b'])=0.523517382413088
Probability(['b', 'd'])=0.0081799591002045
Probability(['a', 'b', 'c'])=0.130879345603272
Probability(['a', 'b', 'd'])=0.032719836400818
Probability(['b', 'c', 'd'])=0.002044989775051125
Probability(['a', 'b', 'c', 'd', 'e'])=0.0081799591002045


In [37]:
for item, prob in Paula.prob_distribution_items.items():
    print(f'Probability({item})={prob}')

Probability(a)=0.8261758691206544
Probability(b)=0.83640081799591
Probability(c)=0.14110429447852763
Probability(d)=0.05112474437627812
Probability(e)=0.0081799591002045
